# Numerical Integration Visualization Dashboard

Numerical integration techniques are used to approximate the definite integral of a function. This dashboard visualizes several common methods and analyzes their convergence properties.

### Methods Summary:
1. **Riemann Sums**: Uses rectangles. Accuracy scales as $O(1/n)$.
2. **Trapezoidal Rule**: Uses linear segments. Accuracy scales as $O(1/n^2)$.
3. **Simpson's Rule**: Uses quadratic segments. Accuracy scales as $O(1/n^4)$.
4. **Gaussian Quadrature**: Uses orthogonal polynomials to achieve high accuracy with few points.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from integrator import NumericalIntegrator

%matplotlib inline
integrator = NumericalIntegrator()

In [15]:
# UI Elements
func_input = widgets.Text(value='sin(x) + 2', description='Function f(x):', style={'description_width': 'initial'})
a_input = widgets.FloatText(value=0.0, description='Lower Bound (a):', style={'description_width': 'initial'})
b_input = widgets.FloatText(value=3.14159, description='Upper Bound (b):', style={'description_width': 'initial'})

# Intervals Controller
n_slider = widgets.IntSlider(value=10, min=1, max=100, step=1, description='Intervals (n):', continuous_update=True, layout={'width': '500px'}, style={'description_width': 'initial'})
n_max_slider = widgets.IntSlider(value=100, min=10, max=2000, step=10, description='Max Intervals Allowed:', style={'description_width': 'initial'})

# Play / Animation widget
play_button = widgets.Play(value=10, min=1, max=100, step=1, interval=100, description="Press play")
widgets.jslink((play_button, 'value'), (n_slider, 'value'))
widgets.jslink((play_button, 'max'), (n_slider, 'max'))

method_dropdown = widgets.Dropdown(
    options=['Riemann Left', 'Riemann Right', 'Riemann Mid', 'Trapezoidal', 'Simpson', 'Gaussian Quadrature'],
    value='Trapezoidal', 
    description='Method:', 
    style={'description_width': 'initial'}
)

show_convergence = widgets.Checkbox(value=False, description='Show Convergence Analysis')
convergence_n_range = widgets.IntRangeSlider(
    value=[2, 512], min=2, max=2048, step=1, 
    description='Convergence n-Range:', 
    style={'description_width': 'initial'}
)

def update_n_max(change):
    n_slider.max = change['new']
    play_button.max = change['new']
    if n_slider.value > n_slider.max:
        n_slider.value = n_slider.max

n_max_slider.observe(update_n_max, names='value')

def render_dashboard(f_str, a, b, n, method, show_conv, conv_range):
    try:
        f, expr, f_int, expr_int = integrator.parse_function(f_str)
        true_area = integrator.get_true_area(f_int, a, b)
        
        if show_conv and method != 'Gaussian Quadrature':
            # Convergence Plot
            n_start, n_end = conv_range
            ns = [2**i for i in range(int(np.log2(n_start)), int(np.log2(n_end)) + 1)]
            if n_start not in ns: ns.insert(0, n_start)
            if n_end not in ns: ns.append(n_end)
            ns = sorted(list(set([val for val in ns if n_start <= val <= n_end])))
            
            res_n, res_err = integrator.get_convergence_data(f, a, b, f_int, ns, method)
            
            fig, ax = plt.subplots(figsize=(12, 6))
            ax.loglog(res_n, res_err, 'o-', label=f'Actual Error ({method})')
            
            if 'Riemann' in method:
                ref = [max(res_err[0], 1e-10) * (res_n[0]/v) for v in res_n]
                ax.loglog(res_n, ref, 'k--', alpha=0.5, label='O(1/n) slope')
            elif 'Mid' in method or 'Trapezoidal' in method:
                ref = [max(res_err[0], 1e-10) * (res_n[0]/v)**2 for v in res_n]
                ax.loglog(res_n, ref, 'k--', alpha=0.5, label='O(1/n^2) slope')
            elif 'Simpson' in method:
                ref = [max(res_err[0], 1e-10) * (res_n[0]/v)**4 for v in res_n]
                ax.loglog(res_n, ref, 'k--', alpha=0.5, label='O(1/n^4) slope')
            
            ax.set_xlabel('Number of intervals (n)')
            ax.set_ylabel('Absolute Error')
            ax.set_title(f'Convergence Analysis for {method}')
            ax.legend(); ax.grid(True, which="both", ls="-", alpha=0.3)
        else:
            # Visualization Plot
            fig, ax = plt.subplots(figsize=(12, 6))
            x_vals = np.linspace(min(a,b) - abs(b-a)*0.1, max(a,b) + abs(b-a)*0.1, 400)
            ax.plot(x_vals, f(x_vals), 'r-', label=f'f(x) = {f_str}', linewidth=2)
            
            approx_area = 0
            h = (b-a)/n if n > 0 else 0
            
            if method == 'Riemann Left':
                approx_area, x_edges, x_pts, y_pts = integrator.riemann_left(f, a, b, n)
                ax.bar(x_pts, y_pts, width=h, align='edge', alpha=0.3, color='blue', edgecolor='k')
            elif method == 'Riemann Right':
                approx_area, x_edges, x_pts, y_pts = integrator.riemann_right(f, a, b, n)
                ax.bar(x_pts, y_pts, width=-h, align='edge', alpha=0.3, color='blue', edgecolor='k')
            elif method == 'Riemann Mid':
                approx_area, x_edges, x_pts, y_pts = integrator.riemann_mid(f, a, b, n)
                ax.bar(x_pts, y_pts, width=h, align='center', alpha=0.3, color='blue', edgecolor='k')
            elif method == 'Trapezoidal':
                approx_area, x, y = integrator.trapezoid(f, a, b, n)
                ax.fill_between(x, 0, y, alpha=0.3, color='blue')
                ax.plot(x, y, 'b-', alpha=0.5)
            elif method == 'Simpson':
                approx_area, x, y = integrator.simpson(f, a, b, n)
                ax.fill_between(x, 0, y, alpha=0.3, color='blue', label="Simpson's parabolas")
            elif method == 'Gaussian Quadrature':
                approx_area, x_pts, y_pts = integrator.gaussian_quadrature(f, a, b, n)
                ax.vlines(x_pts, 0, y_pts, colors='g', linestyles='dashed', alpha=0.5)
                ax.plot(x_pts, y_pts, 'go', label='Quadrature Points')
            
            abs_err, rel_err = integrator.get_error_metrics(approx_area, true_area)
            ax.set_title(f"{method}: Approx = {approx_area:.5f}, True = {true_area:.5f}, Rel Error={rel_err:.2e}%")
            ax.grid(True, alpha=0.3)
        
        plt.show()
    except Exception as e:
        print(f"Dashboard Error: {e}")

interactive_plot = widgets.interactive_output(render_dashboard, {
    'f_str': func_input, 'a': a_input, 'b': b_input, 
    'n': n_slider, 'method': method_dropdown, 
    'show_conv': show_convergence, 'conv_range': convergence_n_range
})

# Layout
header = widgets.Label("--- NUMERICAL INTEGRATION CONTROLS ---")
animation_controls = widgets.HBox([widgets.Label("ANIMATE:"), play_button])
main_ui = widgets.VBox([
    header,
    func_input, 
    widgets.HBox([a_input, b_input]),
    widgets.HBox([method_dropdown, show_convergence]),
    widgets.VBox([widgets.Label("Intervals Control:"), n_slider, n_max_slider]),
    animation_controls,
    convergence_n_range,
    interactive_plot
])

display(main_ui)
